---
layout: post
title: PhiFlow
---

In [1]:
from pathlib import Path
import os
from collections import defaultdict

from IPython.display import HTML, Image
from phi.torch.flow import *
from tqdm.notebook import trange
import matplotlib.pyplot as plt

ROOT = Path("./assets/img/")

if not os.path.exists(ROOT):
    os.makedirs(ROOT)

PhiFlow is a differentiable fluid simulation in python. Let's see a basic [example](https://tum-pbs.github.io/PhiFlow/examples/grids/Smoke_Plume.html).

In [2]:
def clamp_field(f, lo=0.0, hi=1.0):
    v = f.values                     # backend tensor
    v = math.maximum(lo, math.minimum(hi, v))
    return f.with_values(v)

In [9]:
def smoke_plume():

    domain = Box(x=100, y=100)
    
    inflow = Sphere(x=50, y=9.5, radius=5)
    inflow_rate = 0.2

    def step(v, s, p, dt):
        s = advect.mac_cormack(s, v, dt) + inflow_rate * resample(inflow, to=s, soft=True)
        
        buoyancy = resample(s * (0, 0.1), to=v)
        v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
        
        v, p = fluid.make_incompressible(v, (), Solve('auto', 1e-3, x0=p))
        return v, s, p

    v0 = StaggeredGrid(0, 0, domain, x=64, y=64)
    smoke0 = CenteredGrid(0, ZERO_GRADIENT, domain, x=200, y=200)

    v_trj, s_trj, p_trj = iterate(step, batch(time=150), v0, smoke0, None, dt=.5, range=trange, substeps=3)

    anim = plot(s_trj, animate='time', frame_time=80)
    plt.close()

    gif_path = ROOT / "smoke_plume.gif"  
    anim.save(gif_path, writer="pillow", fps=30)
    return Image(url=gif_path)

smoke_plume()

  0%|          | 0/150 [00:00<?, ?it/s]

/app/.venv/lib/python3.12/site-packages/phiml/backend/_linalg.py:345: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  x = spsolve(lin[batch], y[batch])  # returns nan when diverges


<Figure size 640x480 with 0 Axes>